In [1]:
import asyncio
import itertools
import json
import logging
import math
import multiprocessing
import random
import sys
import time
import os
from threading import Thread
from urllib.parse import unquote,quote
import aiohttp
import pandas as pd
import pymongo
import requests
from lxml import etree
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
import hashlib
import csv


In [2]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client['traffic']
metro_station = db['metro_station_v2']
metro_line = db['metro_line_v2']





In [3]:
next(metro_station.find())

{'_id': 'BV10013454',
 'su': '1',
 'udsu': '1;1',
 'n': '苹果园',
 'sid': '110100023339034',
 'r': [{'lid': '900000069871', 'ln': 'S1线'},
  {'lid': '110100023339', 'ln': '6号线'}],
 'udsi': '110100023339034;110100023340035',
 't': '1',
 'si': '110100023339034',
 'sl': '116.178945,39.925686',
 'udli': '110100023339;110100023340',
 'lg': '0',
 'sp': 'PingGuoYuan',
 'lng': 116.178945,
 'lat': 39.925686,
 'cn': '北京市',
 'ftime': '5:51',
 'ltime': '23:32'}

In [4]:
fs= open('station.csv','w',encoding='utf_8_sig',newline='')
csv_w = csv.writer(fs)
col = ['id','name','lng','lat','cityName','lineNames','lineID']
csv_w.writerow(col)
for item in  metro_station.find():
    print(item['r'],item['_id'])
    ln = map(lambda x:x['ln'],item['r'])

    ln = '|'.join(ln)
    lid = list(map(lambda x:x['lid'],item['r']))
    lid = '|'.join(lid)
    row = [item['_id'],item['n'],item['lng'], item['lat'], item['cn'],ln,lid]
    csv_w.writerow(row)
    

[{'lid': '900000069871', 'ln': 'S1线'}, {'lid': '110100023339', 'ln': '6号线'}] BV10013454
[{'lid': '900000069871', 'ln': 'S1线'}, {'lid': '110100023339', 'ln': '6号线'}, {'lid': '900000159716', 'ln': '11号线'}] BV11494889
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813547
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813541
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813543
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813544
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813542
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813546
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10000295
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013455
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10000302
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013456
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013457
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013458
[{'lid': '110100023110', 'ln': '1号线八通线'}, {'lid': '110100023282', 'ln': '10号线'}] BV10000306
[{'lid': '110100023110', 'ln': '1号线八通线'}, {'lid': '110100023116', 'ln': '9号

In [5]:
fs= open('metro_line.csv','w',encoding='utf_8_sig',newline='')
csv_w = csv.writer(fs)
col = ['id','name','cityName','st_station','ed_station']
csv_w.writerow(col)
for item in  metro_line.find():
    st = item['st'][0]['n']
    ed = item['st'][-1]['n']
    
    row = [item['_id'],item['ln'],item['cn'],st, ed]
    csv_w.writerow(row)
fs.close()    

In [6]:
sz_record =  db['sz_metro_records']
ag_l = [{
    '$group': {
        '_id': {
            'card_no': '$card_no'
        },
        'ids': {
            '$addToSet': '$_id'
        }
    }
}, {
    '$unwind': '$ids'
}, {
    '$group': {
        '_id': '$_id',
        'count_id': {
            '$sum': 1
        }
    }
}]
m_idl = []
for item in sz_record.aggregate(ag_l,allowDiskUse=True):
    if item['count_id'] > 1:
        m_idl.append({'card_no':item['_id']['card_no'],'count': item['count_id']})
   


In [7]:
fs= open('person.csv','w',encoding='utf_8_sig',newline='')
csv_w = csv.writer(fs)
col = ['id','count']
csv_w.writerow(col)

for item in m_idl:
    try:
        row = [item['card_no'],item['count']]
    except Exception:
        print(item)
    csv_w.writerow(row)
fs.close()

In [8]:
m_idl[1:10]

[{'card_no': 'AEAAAAAFB', 'count': 2},
 {'card_no': 'AEAAAACFF', 'count': 2},
 {'card_no': 'AEAAAACHG', 'count': 2},
 {'card_no': 'AEAAAAEBF', 'count': 2},
 {'card_no': 'AEAAAAHFB', 'count': 2},
 {'card_no': 'AEAAAAHIC', 'count': 2},
 {'card_no': 'AEAAACAHB', 'count': 2},
 {'card_no': 'AEAAACBHI', 'count': 4},
 {'card_no': 'AEAAACCBI', 'count': 2}]

In [9]:
fs = open('route_record.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
    'id', 'name', 'card_no', 'st_station', 'ed_station', 'st_time', 'ed_time','line_name'
]
ans = []
csv_w.writerow(col)
def process(t_item):
    record_li = []
    for item in sz_record.find({'card_no': t_item['card_no']}):
        record_li.append(item)
    record_sort_li = sorted(record_li,
                            key=lambda x: x['deal_date'],
                            reverse=False)
    if t_item['count'] % 2 != 0:
        record_sort_li = record_sort_li[:-1]
    for i in range(0, len(record_sort_li), 2):
        name = record_sort_li[i]['station'] + '->' + record_sort_li[i + 1]['station']
        _id = record_sort_li[i]['_id']
        card_no = record_sort_li[i]['card_no']
        st_station = record_sort_li[i]['station']
        ed_station = record_sort_li[i + 1]['station']
        st_time = record_sort_li[i]['deal_date']
        ed_time = record_sort_li[i + 1]['deal_date'] 
        line_name = record_sort_li[i]['company_name'] + '->' + record_sort_li[i + 1]['company_name']
        row = [_id,name,card_no,st_station,ed_station,st_time,ed_time,line_name]
        csv_w.writerow(row)
#         print(row)
#         ans.append(row)
        
def run_record():
    with ThreadPoolExecutor(max_workers=500) as executor:
        future_tasks = []
        for t_item in m_idl:
             future_tasks.append(
                    executor.submit(process, t_item))
        wait(future_tasks, return_when=ALL_COMPLETED)

# run_record()
# csv_w.writerows(ans)
fs.close()


In [10]:
df = pd.read_html('https://www.bjsubway.com/station/zjgls/')
df

[   1号线路网车站站间距信息                                                   
            起始站名   目的站名 距离（米） Unnamed: 3_level_1 Unnamed: 4_level_1
 0            古城  八角游乐园  1921                NaN                NaN
 1         八角游乐园    八宝山  1953                NaN                NaN
 2           八宝山    玉泉路  1479                NaN                NaN
 3           玉泉路    五棵松  1810                NaN                NaN
 4           五棵松    万寿路  1778                NaN                NaN
 5           万寿路    公主坟  1313                NaN                NaN
 6           公主坟  军事博物馆  1172                NaN                NaN
 7         军事博物馆    木樨地  1166                NaN                NaN
 8           木樨地   南礼士路  1291                NaN                NaN
 9          南礼士路    复兴门   424                NaN                NaN
 10          复兴门     西单  1596                NaN                NaN
 11           西单   天安门西  1217                NaN                NaN
 12         天安门西   天安门东   925                NaN

In [11]:
[{
    '$group': {
        '_id': {
            'card_no': '$card_no'
        },
        'ids': {
            '$addToSet': '$_ id'
        }
    }
}, {
    '$unwind': '$ids'
}, {
    '$group': {
        '_id': '$_id',
        'count_id': {
            '$sum': 1
        }
    }
}]

[{'$group': {'_id': {'card_no': '$card_no'}, 'ids': {'$addToSet': '$_ id'}}},
 {'$unwind': '$ids'},
 {'$group': {'_id': '$_id', 'count_id': {'$sum': 1}}}]

In [12]:
df = pd.read_html('https://www.bjsubway.com/station/smcsj/')


1号线八通线首末车时刻表                                                        
           车站名称 往环球度假区方向                            往古城方向              
           车站名称     首车时间   末车时间                      首车时间   末车时间       
           车站名称     首车时间     全程         半程四惠东        首车时间   末车时间       
           车站名称     首车时间  周日至周四  周五、周六  周日至周四 周五、周六  首车时间  周日至周四  周五、周六
0           苹果园       ――     ――     ――     ――    ――    ――     ――     ――
1            古城     4:57  22:51  23:52  23:33  0:34    ――     ――     ――
2         八角游乐园     5:00  22:54  23:55  23:36  0:37  5:31   0:17   1:21
3           八宝山     5:03  22:57  23:58  23:39  0:40  5:28   0:14   1:18
4           玉泉路     5:06  22:59   0:00  23:41  0:42  5:25   0:12   1:16
5           五棵松     5:08  23:02   0:03  23:44  0:45  5:23   0:09   1:13
6           万寿路     5:11  23:05   0:06  23:47  0:48  5:20   0:06   1:10
7           公主坟     5:13  23:07   0:08  23:49  0:50  5:17   0:04   1:08
8         军事博物馆     5:16  23:10   0:11  23:52  0:53  5:15   0:01   1:05
9           木樨地     5:18  23:12   0:13  23:54  0:55  5:13  23:59   1:03
10         南礼士路     5:20  23:14   0:15  23:56  0:57  5:11  23:57   1:01
11          复兴门     5:13  23:16   0:17  23:58  0:59  5:08  23:55   0:59
12           西单     5:17  23:19   0:20   0:01  1:02  5:17  23:53   0:57
13         天安门西     5:19  23:21   0:22   0:03  1:04  5:14  23:50   0:54
14         天安门东     5:21  23:23   0:24   0:05  1:06  5:12  23:48   0:52
15          王府井     5:23  23:25   0:26   0:07  1:08  5:10  23:46   0:50
16           东单     5:25  23:27   0:28   0:09  1:10  5:08  23:44   0:48
17          建国门     5:27  23:29   0:30   0:11  1:12  5:06  23:42   0:46
18          永安里     5:30  23:32   0:33   0:14  1:15  5:03  23:39   0:43
19           国贸     5:32  23:34   0:35   0:16  1:17  5:01  23:37   0:41
20          大望路     5:34  23:36   0:37   0:18  1:19  4:59  23:35   0:39
21           四惠     5:37  23:39   0:40   0:21  1:22  4:56  23:32   0:36
22          四惠东     5:40  23:42   0:43     ――    ――  5:04  23:29   0:33
23          高碑店     5:43  23:45   0:46     ――    ――  5:40  23:26   0:30
24         传媒大学     5:46  23:48   0:49     ――    ――  5:38  23:24   0:28
25           双桥     5:48  23:50   0:51     ――    ――  5:35  23:21   0:25
26           管庄     5:51  23:53   0:54     ――    ――  5:32  23:18   0:22
27          八里桥     5:54  23:56   0:57     ――    ――  5:29  23:15   0:19
28         通州北苑     5:57  23:59   1:00     ――    ――  5:26  23:12   0:16
29           果园     6:00   0:02   1:03     ――    ――  5:24  23:10   0:14
30          九棵树     6:02   0:04   1:05     ――    ――  5:22  23:08   0:12
31           梨园     6:04   0:06   1:07     ――    ――  5:19  23:05   0:09
32          临河里     6:06   0:08   1:09     ――    ――  5:17  23:03   0:07
33           土桥     4:59   0:10   1:11     ――    ――  5:15  23:01   0:05
34           花庄     5:02   0:13   1:14     ――    ――  5:12  22:58   0:02
35        环球度假区       ――     ――     ――     ――    ――  5:09  22:55  23:59

In [14]:
df[0].columns

MultiIndex([('1号线八通线首末车时刻表',     '车站名称', '车站名称',  '车站名称',  '车站名称'),
            ('1号线八通线首末车时刻表', '往环球度假区方向', '首车时间',  '首车时间',  '首车时间'),
            ('1号线八通线首末车时刻表', '往环球度假区方向', '末车时间',    '全程', '周日至周四'),
            ('1号线八通线首末车时刻表', '往环球度假区方向', '末车时间',    '全程', '周五、周六'),
            ('1号线八通线首末车时刻表', '往环球度假区方向', '末车时间', '半程四惠东', '周日至周四'),
            ('1号线八通线首末车时刻表', '往环球度假区方向', '末车时间', '半程四惠东', '周五、周六'),
            ('1号线八通线首末车时刻表',    '往古城方向', '首车时间',  '首车时间',  '首车时间'),
            ('1号线八通线首末车时刻表',    '往古城方向', '末车时间',  '末车时间', '周日至周四'),
            ('1号线八通线首末车时刻表',    '往古城方向', '末车时间',  '末车时间', '周五、周六')],
           )

In [15]:
k = 0
for i in  range(len(df)):
    print(i)
    for j in  range(len(df[i])):
        flag = True
        for item in metro_station.find({'n':df[i].iloc[j][0]}):
            k+=1
            item['ftime'] = df[i].iloc[j][1]
            item['ltime'] = df[i].iloc[j][2]
            if i ==  10:
                item['ltime'] = df[i].iloc[j][3]
                
#             print(item['ftime'],item['ltime'])
            metro_station.update({'_id':item['_id']},item,upsert=True)
            flag = False
        if flag:
            print(df[i].columns[0][0],df[i].iloc[j][0])
print(k)

0
1
2
3
4
6号线首末车时刻表 通运门


D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


5
6
7
8
9
10
11
14号线首末车时刻表 陶然桥
14号线首末车时刻表 高家园
12
13
14
15
19号线首末车时刻表 太平桥
19号线首末车时刻表 北太平庄
16
17
18
19
20
21
22
23
477


In [16]:
df

[   1号线八通线首末车时刻表                                                        
            车站名称 往环球度假区方向                            往古城方向              
            车站名称     首车时间   末车时间                      首车时间   末车时间       
            车站名称     首车时间     全程         半程四惠东        首车时间   末车时间       
            车站名称     首车时间  周日至周四  周五、周六  周日至周四 周五、周六  首车时间  周日至周四  周五、周六
 0           苹果园       ――     ――     ――     ――    ――    ――     ――     ――
 1            古城     4:57  22:51  23:52  23:33  0:34    ――     ――     ――
 2         八角游乐园     5:00  22:54  23:55  23:36  0:37  5:31   0:17   1:21
 3           八宝山     5:03  22:57  23:58  23:39  0:40  5:28   0:14   1:18
 4           玉泉路     5:06  22:59   0:00  23:41  0:42  5:25   0:12   1:16
 5           五棵松     5:08  23:02   0:03  23:44  0:45  5:23   0:09   1:13
 6           万寿路     5:11  23:05   0:06  23:47  0:48  5:20   0:06   1:10
 7           公主坟     5:13  23:07   0:08  23:49  0:50  5:17   0:04   1:08
 8         军事博物馆     5:16  23:10   0:11  23:52  0:5

In [17]:

adjoin = db['adjoin']


In [18]:
import hashlib


In [22]:
fs = open('adjoin_rela.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
   'id', 'pid1','pid2','n1','n2','distance','ln'
]
csv_w.writerow(col)
j = 1 
for i in range(len(df)):
    lname = df[i].columns[0][0].split('路网')[0]
    print(i)
    d = 0
    flag= True
    for k in range(len(df[i])):

        n1 = df[i].iloc[k][0]
        n2 = df[i].iloc[k][1]
        try:
            distance = d + int(df[i].iloc[k][2])
        except Exception as e:
            continue
        pid1 = '-'
        pid2 = '-'
        for item in metro_station.find({'n': n1}):
            pid1 = item['_id']
        for item in metro_station.find({'n': n2}):
            pid2 = item['_id']
        if pid2 == '-':
            d = distance
            flag = False
        else:
            fn2 = n2
            fpid2 = pid2
            
        if pid1 == '-':
            d = 0
            flag = True
        else:
            fn1 = n1
            fpid1 = pid1
        a = hashlib.md5(("%s-%s"%(fpid1,fpid2)).encode('utf-8'))
        md5 = a.hexdigest()
        if flag:        
            item = {}
            item['_id'] = j
            item['pid1'] = fpid1
            item['pid2'] = fpid2
            item['n1'] = fn1
            item['n2'] = fn2
            item['dist'] = distance
            item['ln'] = lname
            j += 1
            csv_w.writerow(list(item.values()))
#             adjoin.update({'_id': item['_id']}, item, upsert=True)
            
            
fs.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [82]:
list(item.values())

['310bb3d4f09c21802dd33ed712884fa4',
 'BV10009415',
 'BV10000199',
 '3号航站楼',
 '三元桥',
 18322,
 '首都机场线']

In [59]:
df[11]

14号线路网车站站间距信息                                                   
            起始站名   目的站名 距离（米） Unnamed: 3_level_1 Unnamed: 4_level_1
0            张郭庄    园博园  1345                NaN                NaN
2            园博园    大瓦窑  4073                NaN                NaN
4            大瓦窑    郭庄子  1236                NaN                NaN
6            郭庄子     大井  2044                NaN                NaN
8             大井    七里庄  1579                NaN                NaN
10           七里庄     西局   845                NaN                NaN
12          北京南站    陶然桥   887                NaN                NaN
14           陶然桥   永定门外  1063                NaN                NaN
16          永定门外     景泰  1119                NaN                NaN
18            景泰    蒲黄榆  1025                NaN                NaN
20           蒲黄榆     方庄  1486                NaN                NaN
22            方庄    十里河  1618                NaN                NaN
24           十里河   南八里庄  1147                NaN                NaN
26          南八里庄  北工大西门  1276                NaN                NaN
28         北工大西门    平乐园  1128                NaN                NaN
30           平乐园    九龙山   897                NaN                NaN
32           九龙山    大望路  1780                NaN                NaN
34           大望路     红庙   708                NaN                NaN
36            红庙    金台路   894                NaN                NaN
38           金台路   朝阳公园  1085                NaN                NaN
40          朝阳公园     枣营  1221                NaN                NaN
42            枣营   东风北桥  2173                NaN                NaN
44          东风北桥     将台  1600                NaN                NaN
46            将台    高家园  1171                NaN                NaN
48           高家园    望京南   676                NaN                NaN
50           望京南     阜通  1168                NaN                NaN
52            阜通     望京   903                NaN                NaN
54            望京    东湖渠  1283                NaN                NaN
56           东湖渠    来广营  1100                NaN                NaN
58           来广营    善各庄  1364                NaN                NaN

In [4]:
ids = []
k = 1
fs = open('adjoin_rela_v2.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
    'pid1','no','pid2','id','distance',
]
csv_w.writerow(col)



for item in metro_line.find():
    st_l = item['st']
    for i in range(len(st_l)-1):
        csv_w.writerow([st_l[i]['poiid'],str(i+1),st_l[i + 1]['poiid'],str(k),])
        k+=1
        
fs.close()
